In [15]:
import requests
import numpy as np
import pandas as pd
from transformers import pipeline
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report



# Load Data

In [16]:
# Load the CSV file into a DataFrame
df_es = pd.read_csv('https://raw.githubusercontent.com/svisel22/SS23-BIPM-Analytics-Lab---Group-4-repository/main/Preprocessing/data_clean/labeled-data/labeled-es_clean_1-1.csv', sep=';')

In [17]:
df_es.head()

,data,player,language,publishedAt,Label
0,adeyemi firmo el que es su primer gol en lo qu...,palacios,es,2023-01-29T18:25:03Z,NaN
1,"el club aleman, que siempre se ha caracterizad...",palacios,es,2023-01-31T20:41:38Z,NaN
2,"alberto fernandez el presidente de la afa, cl...",palacios,es,2023-02-09T18:32:38Z,NaN
3,alberto fernandez tambien participaron los ot...,palacios,es,2023-02-12T21:13:55Z,NaN
4,fue el momento en que desde las tribunas se de...,palacios,es,2023-02-13T01:05:15Z,NaN


### Fix NA

In [18]:
#for DE datasate drop empty lines
df_es = df_es.dropna(subset=['data'])

# Sentiment analysis full text

In [19]:
sentiment_classifier1 = pipeline('sentiment-analysis', model='finiteautomata/beto-sentiment-analysis')

In [20]:
sentiment_classifier2 = pipeline('sentiment-analysis', model='dccuchile/bert-base-spanish-wwm-uncased')

Some weights of the model checkpoint at dccuchile/bert-base-spanish-wwm-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuc

In [21]:
'''
def perform_comparison_sentiment_analysis(df, sentiment_classifier):
    # Create empty lists to store the sentiment scores and labels
    sentiment_scores = []
    sentiment_labels = []

    # Iterate over the 'data' column in the DataFrame
    for text in df['data']:
        # Perform sentiment analysis using the Hugging Face pipeline
        result = sentiment_classifier(text)[0]
        sentiment_score = result['score']
        
        # Convert logits to predicted label (positive/negative)
        predicted_label = 1 if sentiment_score > 0.5 else 0
        sentiment_label = "positiv" if predicted_label == 1 else "negativ"

        # Append the sentiment score and label to the respective lists
        sentiment_scores.append(sentiment_score)
        sentiment_labels.append(sentiment_label)

    # Add the sentiment scores and labels as new columns in the DataFrame
    df['sentiment_score'] = sentiment_scores
    df['sentiment_label'] = sentiment_labels

    return df
'''
#three labels:

def perform_comparison_sentiment_analysis(df, sentiment_classifier):
    # Create empty lists to store the sentiment scores and labels
    sentiment_scores = []
    sentiment_labels = []

    # Iterate over the 'data' column in the DataFrame
    for text in df['data']:
        # Perform sentiment analysis using the Hugging Face pipeline
        result = sentiment_classifier(text)[0]
        sentiment_score = result['score']
        
        # Determine sentiment label based on sentiment score
        if sentiment_score > 0.6:
            sentiment_label = "positive"
        elif sentiment_score < 0.4:
            sentiment_label = "negative"
        else:
            sentiment_label = "neutral"

        # Append the sentiment score and label to the respective lists
        sentiment_scores.append(sentiment_score)
        sentiment_labels.append(sentiment_label)

    # Add the sentiment scores and labels as new columns in the DataFrame
    df['sentiment_score'] = sentiment_scores
    df['sentiment_label'] = sentiment_labels

    return df


# Perform sentiment analysis on the DataFrame
df1 = perform_comparison_sentiment_analysis(df_es, sentiment_classifier1)

# Print the updated DataFrame
df1.head()



,data,player,language,publishedAt,Label,sentiment_score,sentiment_label
0,adeyemi firmo el que es su primer gol en lo qu...,palacios,es,2023-01-29T18:25:03Z,NaN,0.468314,neutral
1,"el club aleman, que siempre se ha caracterizad...",palacios,es,2023-01-31T20:41:38Z,NaN,0.996520,positive
2,"alberto fernandez el presidente de la afa, cl...",palacios,es,2023-02-09T18:32:38Z,NaN,0.926711,positive
3,alberto fernandez tambien participaron los ot...,palacios,es,2023-02-12T21:13:55Z,NaN,0.961068,positive
4,fue el momento en que desde las tribunas se de...,palacios,es,2023-02-13T01:05:15Z,NaN,0.988847,positive


In [24]:
# Calculate the accuracy
accuracy = (df1['sentiment_label'] == df1['Label']).mean() * 100

# Print the accuracy
print("Accuracy: {:.2f}%".format(accuracy))

Accuracy: 4.22%


In [22]:
# Perform sentiment analysis on the DataFrame
df2 = perform_comparison_sentiment_analysis(df_es, sentiment_classifier2)

# Print the updated DataFrame
df2.head()

,data,player,language,publishedAt,Label,sentiment_score,sentiment_label
0,adeyemi firmo el que es su primer gol en lo qu...,palacios,es,2023-01-29T18:25:03Z,NaN,0.533805,neutral
1,"el club aleman, que siempre se ha caracterizad...",palacios,es,2023-01-31T20:41:38Z,NaN,0.546071,neutral
2,"alberto fernandez el presidente de la afa, cl...",palacios,es,2023-02-09T18:32:38Z,NaN,0.534002,neutral
3,alberto fernandez tambien participaron los ot...,palacios,es,2023-02-12T21:13:55Z,NaN,0.510552,neutral
4,fue el momento en que desde las tribunas se de...,palacios,es,2023-02-13T01:05:15Z,NaN,0.527243,neutral


In [23]:
# Calculate the accuracy
accuracy = (df2['sentiment_label'] == df2['Label']).mean() * 100

# Print the accuracy
print("Accuracy: {:.2f}%".format(accuracy))

Accuracy: 4.22%


## Model 1

In [ ]:
# Perform sentiment analysis on the DataFrame
df = perform_comparison_sentiment_analysis(df_es, sentiment_classifier1)

# Print the updated DataFrame
df.head()

,data,player,language,publishedAt,Label,sentiment_score,sentiment_label
0,adeyemi firmo el que es su primer gol en lo qu...,palacios,es,2023-01-29T18:25:03Z,NaN,0.468314,negativ
1,"el club aleman, que siempre se ha caracterizad...",palacios,es,2023-01-31T20:41:38Z,NaN,0.996520,positiv
2,"alberto fernandez el presidente de la afa, cl...",palacios,es,2023-02-09T18:32:38Z,NaN,0.926711,positiv
3,alberto fernandez tambien participaron los ot...,palacios,es,2023-02-12T21:13:55Z,NaN,0.961068,positiv
4,fue el momento en que desde las tribunas se de...,palacios,es,2023-02-13T01:05:15Z,NaN,0.988847,positiv


In [ ]:
df[df['sentiment_label'] == 'negativ'].shape[0]/df.shape[0]

0.004219409282700422

In [ ]:
print(df['sentiment_label'].unique())
print(df['sentiment_score'].min())
print(df['sentiment_score'].max())

['negativ' 'positiv']
0.46831411123275757
0.9988766312599182


In [ ]:
# Drop rows where 'Label' is NaN or empty
df_es.dropna(subset=['Label'], inplace=True)

# Print the updated DataFrame
data = df_es[df_es['Label'] != 'neutral']

df_es

,data,player,language,publishedAt,Label,sentiment_score,sentiment_label
6,"wirtz, a la derecha, celebra su gol, el primer...",palacios,es,2023-02-23T20:37:20Z,positiv,0.555995,positiv
8,"en tanto, palacios volvio al gol luego de sei...",palacios,es,2023-02-23T23:30:42Z,positiv,0.922271,positiv
18,"panama y curazao con, como era previsible, los...",palacios,es,2023-03-11T17:05:53Z,neutral,0.672797,positiv
28,palacios marco este domingo dos goles ante ba...,palacios,es,2023-03-19T20:19:09Z,positiv,0.623142,positiv
48,"luego ingresaron paulo dybala, juan foyth, rod...",palacios,es,2023-03-29T02:18:28Z,neutral,0.997254,positiv
52,"palacios, el campeon de la seleccion argentina...",palacios,es,2023-04-06T12:09:42Z,negativ,0.982327,positiv
53,"en sus historias de instagram, frias dejo entr...",palacios,es,2023-04-07T23:07:30Z,negativ,0.981422,positiv
54,luego de la publicacion que hizo yesica frias...,palacios,es,2023-04-08T14:28:00Z,negativ,0.998399,positiv
58,alexis mac allister intentara acercar a bright...,palacios,es,2023-04-29T03:41:10Z,neutral,0.979650,positiv
78,"ademas, alphonso davies aparecio para despejar...",frimpong,es,2023-03-19T21:50:51Z,positiv,0.998621,positiv


In [ ]:
data

,data,player,language,publishedAt,Label,sentiment_score,sentiment_label
6,"wirtz, a la derecha, celebra su gol, el primer...",palacios,es,2023-02-23T20:37:20Z,positiv,0.555995,positiv
8,"en tanto, palacios volvio al gol luego de sei...",palacios,es,2023-02-23T23:30:42Z,positiv,0.922271,positiv
28,palacios marco este domingo dos goles ante ba...,palacios,es,2023-03-19T20:19:09Z,positiv,0.623142,positiv
52,"palacios, el campeon de la seleccion argentina...",palacios,es,2023-04-06T12:09:42Z,negativ,0.982327,positiv
53,"en sus historias de instagram, frias dejo entr...",palacios,es,2023-04-07T23:07:30Z,negativ,0.981422,positiv
54,luego de la publicacion que hizo yesica frias...,palacios,es,2023-04-08T14:28:00Z,negativ,0.998399,positiv
78,"ademas, alphonso davies aparecio para despejar...",frimpong,es,2023-03-19T21:50:51Z,positiv,0.998621,positiv
80,frimpong\xaaparecio dentro del area para defin...,frimpong,es,2023-04-01T16:20:30Z,positiv,0.900498,positiv
82,"el delantero nigeriano eludio rivales, se cayo...",frimpong,es,2023-04-13T21:28:24Z,positiv,0.867035,positiv
100,"sardar azmoun empato el partido, a los minuto...",bakker,es,2023-02-26T17:11:43Z,positiv,0.819514,positiv


In [ ]:
# Calculate the accuracy
accuracy = (data['sentiment_label'] == data['Label']).mean() * 100

# Print the accuracy
print("Accuracy: {:.2f}%".format(accuracy))

Accuracy: 50.00%


In [ ]:
# Assuming you have the true labels in 'Label' column and predicted labels in 'sentiment_label' column of the DataFrame
true_labels = data['Label']
predicted_labels = data['sentiment_label']

# Create the confusion matrix
cm = confusion_matrix(true_labels, predicted_labels)

# Convert the confusion matrix to a DataFrame for better visualization
labels = np.unique(np.concatenate((true_labels, predicted_labels)))
cm_df = pd.DataFrame(cm, index=labels, columns=labels)

# Display the confusion matrix
print("Confusion Matrix:")
print(cm_df)

Confusion Matrix:
         negativ  positiv
negativ        0       10
positiv        0       10


In [ ]:
# Assuming you have the true labels in 'Label' column and predicted labels in 'sentiment_label' column of the DataFrame
true_labels = data['Label']
predicted_labels = data['sentiment_label']

# Generate the classification report
report = classification_report(true_labels, predicted_labels)

# Display the classification report
print("Classification Report:")
print(report)


Classification Report:
              precision    recall  f1-score   support

     negativ       0.00      0.00      0.00        10
     positiv       0.50      1.00      0.67        10

    accuracy                           0.50        20
   macro avg       0.25      0.50      0.33        20
weighted avg       0.25      0.50      0.33        20



C:\Users\joana\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\joana\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\joana\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1344

## Model 2

In [ ]:
# Perform sentiment analysis on the DataFrame
df = perform_comparison_sentiment_analysis(df_es, sentiment_classifier2)

# Print the updated DataFrame
df.head()

,data,player,language,publishedAt,Label,sentiment_score,sentiment_label
6,"wirtz, a la derecha, celebra su gol, el primer...",palacios,es,2023-02-23T20:37:20Z,positiv,0.546026,positiv
8,"en tanto, palacios volvio al gol luego de sei...",palacios,es,2023-02-23T23:30:42Z,positiv,0.570668,positiv
18,"panama y curazao con, como era previsible, los...",palacios,es,2023-03-11T17:05:53Z,neutral,0.539131,positiv
28,palacios marco este domingo dos goles ante ba...,palacios,es,2023-03-19T20:19:09Z,positiv,0.565715,positiv
48,"luego ingresaron paulo dybala, juan foyth, rod...",palacios,es,2023-03-29T02:18:28Z,neutral,0.588336,positiv


In [ ]:
df[df['sentiment_label'] == 'negativ'].shape[0]/df.shape[0]

0.0

In [ ]:
# Drop rows where 'Label' is NaN or empty
df_es.dropna(subset=['Label'], inplace=True)

# Print the updated DataFrame
data = df_es[df_es['Label'] != 'neutral']

df_es

,data,player,language,publishedAt,Label,sentiment_score,sentiment_label
6,"wirtz, a la derecha, celebra su gol, el primer...",palacios,es,2023-02-23T20:37:20Z,positiv,0.546026,positiv
8,"en tanto, palacios volvio al gol luego de sei...",palacios,es,2023-02-23T23:30:42Z,positiv,0.570668,positiv
18,"panama y curazao con, como era previsible, los...",palacios,es,2023-03-11T17:05:53Z,neutral,0.539131,positiv
28,palacios marco este domingo dos goles ante ba...,palacios,es,2023-03-19T20:19:09Z,positiv,0.565715,positiv
48,"luego ingresaron paulo dybala, juan foyth, rod...",palacios,es,2023-03-29T02:18:28Z,neutral,0.588336,positiv
52,"palacios, el campeon de la seleccion argentina...",palacios,es,2023-04-06T12:09:42Z,negativ,0.529579,positiv
53,"en sus historias de instagram, frias dejo entr...",palacios,es,2023-04-07T23:07:30Z,negativ,0.503963,positiv
54,luego de la publicacion que hizo yesica frias...,palacios,es,2023-04-08T14:28:00Z,negativ,0.535089,positiv
58,alexis mac allister intentara acercar a bright...,palacios,es,2023-04-29T03:41:10Z,neutral,0.570315,positiv
78,"ademas, alphonso davies aparecio para despejar...",frimpong,es,2023-03-19T21:50:51Z,positiv,0.510269,positiv


In [ ]:
data

,data,player,language,publishedAt,Label,sentiment_score,sentiment_label
6,"wirtz, a la derecha, celebra su gol, el primer...",palacios,es,2023-02-23T20:37:20Z,positiv,0.546026,positiv
8,"en tanto, palacios volvio al gol luego de sei...",palacios,es,2023-02-23T23:30:42Z,positiv,0.570668,positiv
28,palacios marco este domingo dos goles ante ba...,palacios,es,2023-03-19T20:19:09Z,positiv,0.565715,positiv
52,"palacios, el campeon de la seleccion argentina...",palacios,es,2023-04-06T12:09:42Z,negativ,0.529579,positiv
53,"en sus historias de instagram, frias dejo entr...",palacios,es,2023-04-07T23:07:30Z,negativ,0.503963,positiv
54,luego de la publicacion que hizo yesica frias...,palacios,es,2023-04-08T14:28:00Z,negativ,0.535089,positiv
78,"ademas, alphonso davies aparecio para despejar...",frimpong,es,2023-03-19T21:50:51Z,positiv,0.510269,positiv
80,frimpong\xaaparecio dentro del area para defin...,frimpong,es,2023-04-01T16:20:30Z,positiv,0.539357,positiv
82,"el delantero nigeriano eludio rivales, se cayo...",frimpong,es,2023-04-13T21:28:24Z,positiv,0.502849,positiv
100,"sardar azmoun empato el partido, a los minuto...",bakker,es,2023-02-26T17:11:43Z,positiv,0.527136,positiv


In [ ]:
# Calculate the accuracy
accuracy = (data['sentiment_label'] == data['Label']).mean() * 100

# Print the accuracy
print("Accuracy: {:.2f}%".format(accuracy))

Accuracy: 50.00%


In [ ]:
data.head()

,data,player,language,publishedAt,Label,sentiment_score,sentiment_label
6,"wirtz, a la derecha, celebra su gol, el primer...",palacios,es,2023-02-23T20:37:20Z,positiv,0.546026,positiv
8,"en tanto, palacios volvio al gol luego de sei...",palacios,es,2023-02-23T23:30:42Z,positiv,0.570668,positiv
28,palacios marco este domingo dos goles ante ba...,palacios,es,2023-03-19T20:19:09Z,positiv,0.565715,positiv
52,"palacios, el campeon de la seleccion argentina...",palacios,es,2023-04-06T12:09:42Z,negativ,0.529579,positiv
53,"en sus historias de instagram, frias dejo entr...",palacios,es,2023-04-07T23:07:30Z,negativ,0.503963,positiv


In [ ]:
# Assuming you have the true labels in 'Label' column and predicted labels in 'sentiment_label' column of the DataFrame
true_labels = data['Label']
predicted_labels = data['sentiment_label']

# Create the confusion matrix
cm = confusion_matrix(true_labels, predicted_labels)

# Convert the confusion matrix to a DataFrame for better visualization
labels = np.unique(np.concatenate((true_labels, predicted_labels)))
cm_df = pd.DataFrame(cm, index=labels, columns=labels)

# Display the confusion matrix
print("Confusion Matrix:")
print(cm_df)

Confusion Matrix:
         negativ  positiv
negativ        0       10
positiv        0       10


In [ ]:
# Assuming you have the true labels in 'Label' column and predicted labels in 'sentiment_label' column of the DataFrame
true_labels = data['Label']
predicted_labels = data['sentiment_label']

# Generate the classification report
report = classification_report(true_labels, predicted_labels)

# Display the classification report
print("Classification Report:")
print(report)


Classification Report:
              precision    recall  f1-score   support

     negativ       0.00      0.00      0.00        10
     positiv       0.50      1.00      0.67        10

    accuracy                           0.50        20
   macro avg       0.25      0.50      0.33        20
weighted avg       0.25      0.50      0.33        20



C:\Users\joana\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\joana\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\joana\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1344